# Combine the differnet data sources

1. Data X from business news and business cycles
2. Data Y from the MSCI Website
3. risk-free rate to compute the excess returns


In [ ]:
import calculate_returns
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
# get the text data
df_text = pd.read_csv(
    "/Users/simonleiner/PycharmProjects/bachelorarbeit/csv_files/Monthly_Topic_Attention_Theta.csv",
    header=0,
    sep=",",
    thousands=None,
)

# adjust the date column
df_text["date"] = pd.to_datetime(df_text["date"], errors="raise")

# set the date column as the index
df_text = df_text.set_index("date")

# Note: 1984 at the top, ascending
display(df_text)

In [ ]:
# get the sp500 data
df_sp500 = pd.read_csv(
    "/Users/simonleiner/PycharmProjects/bachelorarbeit/csv_files/MULTPL-SP500_REAL_PRICE_MONTH.csv",
    header=0,
    sep=",",
    thousands=None,
)

# adjust the date column
df_sp500["Date"] = pd.to_datetime(df_sp500["Date"], errors="raise")

# set the date column as the index
df_sp500 = df_sp500.set_index("Date")

# data is in reverse !! 2017 at the top so sort!!!
df_sp500 = df_sp500.sort_index(ascending=True)

# Note 403 datapoints, we will loose the first datapoint when calculating the reuturns

# plotting
sns.lineplot(data=df_sp500["Value"])
plt.show()

In [ ]:
# calculate the returns: loose first datapoint
df_sp500 = calculate_returns.calculate_log_returns(df_sp500, "Value")

# plotting
sns.lineplot(data=df_sp500["return"])
plt.show()

# save the returns

In [ ]:
# get the 3 month us treasury bill rate in %
df_rate = pd.read_csv(
    "/Users/simonleiner/PycharmProjects/bachelorarbeit/csv_files/F-F_Research_Data_Factors.CSV",
    header=0,
    sep=",",
    thousands=None,
)

# only get the relevant data
df_rate = df_rate.drop(["date", "Mkt-RF", "SMB", "HML"], axis=1)

# set the index accordingly
df_rate = df_rate.set_index(df_sp500.index)

# can't be in decimlas, because we would by 0.5 have a risk free rate of 50%
# https://fred.stlouisfed.org/series/DGS1MO -> must divide by 100
df_rate = df_rate / 100

# plotting
sns.lineplot(data=df_rate["RF"])
plt.show()

# save the risk free rate

In [ ]:
# calculate the excess returns
df_sp500["exc_return"] = df_sp500["return"] - df_rate["RF"]

In [ ]:
# we want to forecast the next month with the data availalb eto this point in time !!!

# changed in the csv files : 1-3, 5 -7, 6 - 8

# set the index accordingly: we take the index of teh asset prices!!
df_text = df_text.set_index(df_sp500.index)

In [ ]:
# combine the 2 dataframes: Take the datetime index from the text data
df_combined = pd.concat([df_text, df_sp500], axis=1)

# delete added columns
df_combined = df_combined.drop(["Value", "return"], axis=1)


# plotting
sns.lineplot(data=df_combined["exc_return"])
plt.show()